# Harry Potter Characters Relationship Networking
Project Idea - Thu Vu Data Analytics.
#### This project revolves around web scrapping and network analysis.

## Data Mining and Cleaning
### Import Libraries
First we will be importing the libraries.\
We will be using selenium for web scrapping in thsi project, other options beautiful soap, scrapy(for large projects)
* Selenium WebDriver is a web framework that permits you to execute cross-browser tests. This tool is used for automating web-based application testing to verify that it performs expectedly.
* webdriver_manager - is reuired to install the ChromeDriverManager(), which is used to open the chrome app

In [3]:
import pandas as pd
import numpy as np
import spacy as sp
# pip install selenium
from selenium import webdriver
# pip install webdriver_manager
from webdriver_manager.chrome import ChromeDriverManager
import warnings
import pickle
warnings.filterwarnings('ignore')

In [21]:
driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - Downloading: 100%|████████████████████████████████████████████████████████████████████████████████████████| 6.68M/6.68M [05:03<00:00, 23.1kB/s]


## * Jump to Web Scrapping Part 2 incase you are trying to execute this project
### Web Scrapping 


* The interesting part, we specify the url from which we would like to scrape content, and suggest the driver to open it.
* Next after the inspecting the web page I identified that the webpage had the characters seperated in different web pages alphabetical.
* So I created a list of links for each alphabet.
* Next we get the elements(character names) by the itemprop of the span tags and their respective description by the class name of the div tag. For this we first inspect the web page to identify the any specific identifier to get the elements. Add the character name along with its respective desciption into a description. Then we convert it into dataframe, which will be helpful in further cleaning and wrangling.

In [3]:
page_url = "https://www.hp-lexicon.org/characters/#index_of_characters"

driver.get(page_url)

In [23]:
from selenium.webdriver.common.by import By

In [5]:
def listAlphabet():
  return [chr(i) for i in range(ord('A'),ord('Z')+1)]

atoz = listAlphabet()

In [6]:
alpha_links = []
for i in atoz:
    l = "https://www.hp-lexicon.org/character/?letter="+i
    alpha_links.append(l)

In [ ]:
alpha_links

In [19]:
characters = []
for link in alpha_links:
    driver.get(link)
    chars = driver.find_elements(By.XPATH, "//span[@itemprop='headline']")
    descs = driver.find_elements(By.CLASS_NAME,"descr")
    for ch,d in zip(chars,descs):
        characters.append({'char_name':ch.text,'description':d.text})

In [21]:
hp_characters = pd.DataFrame(characters)

In [22]:
hp_characters.head(5)

,char_name,description
0,Mrs. Abbott,"Mrs. Abbott was the mother of Hannah Abbott, a..."
1,Abbott family,The Abbott family is a wizarding family that o...
2,Hannah Abbott,Hannah Abbott is a Hufflepuff student in Harry...
3,Abel Treetops,Abel Treetops was a wizard from Cincinnati who...
4,Euan Abercrombie,Euan Abercrombie was a small boy with prominen...


In [23]:
hp_characters.to_csv('character_details.csv',index=False)

In [4]:
chars = pd.read_csv('character_details.csv')
chars

,char_name,description
0,Mrs. Abbott,"Mrs. Abbott was the mother of Hannah Abbott, a..."
1,Abbott family,The Abbott family is a wizarding family that o...
2,Hannah Abbott,Hannah Abbott is a Hufflepuff student in Harry...
3,Abel Treetops,Abel Treetops was a wizard from Cincinnati who...
4,Euan Abercrombie,Euan Abercrombie was a small boy with prominen...
...,...,...
2015,Georgi Zdravko,Georgi Zdravko played Keeper for the Bulgarian...
2016,Zograf,Zograf played Keeper for the Bulgarian Nationa...
2017,Zonko,Founder(?) of Zonko’s Joke Shop. Possibly a re...
2018,Valentina Vázquez,Valentina Vázquez was President of the Argenti...


### Data Preprocessing
###### import re #(regex-regular expressions)

* First I create a sepearate column to strore the bracketed info from the character name, and removed it from the character name
* Next I removed rows without descriptions
* Then dropped duplicate rows
* Next I dropped rows, where the character name had more than 4 elements or the starting alphabet was lower
* Then I did a few modifications for character names containing the/The
* After all this I gave up, because I thought of a better approach, a more simple one to be honest

In [4]:
import re

In [44]:
extra = []
for i in range(chars.shape[0]):
    s = chars.iloc[i]['char_name'].find('(')
    if s>0:
        t = chars.iloc[i]['char_name'].split(' ')
        extra.append(t[-1])
    else:
        extra.append('')

In [46]:
chars['extra'] = extra

In [57]:
chars.to_csv('character_details.csv',index=False)

In [ ]:
chars =  pd.read_csv('character_details.csv')

In [59]:
chars['char_name'] = chars['char_name'].apply(lambda x:re.sub("[\(].*?[\)]","",x))

In [65]:
chars.to_csv('character_details.csv',index=False)

In [3]:
chars =  pd.read_csv('character_details.csv')

In [5]:
chars = chars[chars['description'].notnull()]

In [6]:
chars.reset_index(inplace = True, drop = True)

In [8]:
chars[chars.duplicated()]

,char_name,description,extra
152,Ali Bashir,Ali Bashir was a flying carpet merchant who wa...,NaN
171,Amy Benson,Amy Benson was a child from the orphanage whom...,NaN
179,Bertram Aubrey,"Hogwarts student, 1970s James Potter and Siriu...",NaN
190,Alphard Black,Alphard Black was an uncle to Sirius and Regul...,NaN
191,Annis Black,Annis Black was a hag whose letter is publishe...,NaN
...,...,...,...
2013,Zamira Gulch,Zamira Gulch was the author of Practical House...,NaN
2014,Ladislaw Zamojski,Ladislaw Zamojski is a talented Chaser on the ...,NaN
2015,Georgi Zdravko,Georgi Zdravko played Keeper for the Bulgarian...,NaN
2018,Valentina Vázquez,Valentina Vázquez was President of the Argenti...,NaN


In [13]:
chars.loc[chars.char_name=='Ali Bashir']

,char_name,description,extra
40,Ali Bashir,Ali Bashir was a flying carpet merchant who wa...,NaN
152,Ali Bashir,Ali Bashir was a flying carpet merchant who wa...,NaN


In [7]:
len(chars[chars.duplicated()])

595

In [7]:
chars.drop_duplicates(inplace=True, ignore_index=True)

In [28]:
chars

,char_name,description,extra
0,Mrs. Abbott,"Mrs. Abbott was the mother of Hannah Abbott, a...",NaN
1,Abbott family,The Abbott family is a wizarding family that o...,NaN
2,Hannah Abbott,Hannah Abbott is a Hufflepuff student in Harry...,NaN
3,Abel Treetops,Abel Treetops was a wizard from Cincinnati who...,NaN
4,Euan Abercrombie,Euan Abercrombie was a small boy with prominen...,NaN
...,...,...,...
1420,two young witches on toy broomsticks,NaN,NaN
1421,Yusuf Kama,Yusuf Kama was a French wizard of Senegalese a...,NaN
1422,Yvonne,Yvonne was a friend of Petunia Dursley who was...,NaN
1423,Zograf,Zograf played Keeper for the Bulgarian Nationa...,NaN


In [8]:
nt = []
for i in range(chars.shape[0]):
    k = chars.iloc[i].char_name.split()
    if (len(k)>=3 and k[0][0].islower()) or len(k)>4:
        nt.append(i)

In [9]:
chars.drop(chars.index[nt],inplace=True)

In [10]:
chars.reset_index(drop=True)

,char_name,description,extra
0,Mrs. Abbott,"Mrs. Abbott was the mother of Hannah Abbott, a...",NaN
1,Abbott family,The Abbott family is a wizarding family that o...,NaN
2,Hannah Abbott,Hannah Abbott is a Hufflepuff student in Harry...,NaN
3,Abel Treetops,Abel Treetops was a wizard from Cincinnati who...,NaN
4,Euan Abercrombie,Euan Abercrombie was a small boy with prominen...,NaN
...,...,...,...
1223,Young Red-Haired Witch,"In early September of 1927, a young red-haired...",NaN
1224,Yusuf Kama,Yusuf Kama was a French wizard of Senegalese a...,NaN
1225,Yvonne,Yvonne was a friend of Petunia Dursley who was...,NaN
1226,Zograf,Zograf played Keeper for the Bulgarian Nationa...,NaN


In [11]:
nt = []
for i in range(chars.shape[0]):
    k = chars.iloc[i].char_name.split()
    if len(k)>=3 and k[0]=='The':
        chars.iloc[i].char_name = ' '.join(k[1:])

In [12]:
for i in range(chars.shape[0]):
    k = chars.iloc[i].char_name.split()
    if len(k)>=3 and k[1]=='the':
        chars.iloc[i].char_name = k[0]

In [13]:
# for i in range(chars.shape[0]):
#     k = chars.iloc[i].char_name.split()
#     if len(k)>=3:
#         print(chars.iloc[i].char_name)
chars.loc[chars.char_name=='Mr. and Mrs. Bones']

,char_name,description,extra
227,Mr. and Mrs. Bones,Mr. and Mrs. Bones were a couple killed by Dea...,NaN


In [14]:
chars['firstname'] = None
chars

,char_name,description,extra,firstname
0,Mrs. Abbott,"Mrs. Abbott was the mother of Hannah Abbott, a...",NaN,None
1,Abbott family,The Abbott family is a wizarding family that o...,NaN,None
2,Hannah Abbott,Hannah Abbott is a Hufflepuff student in Harry...,NaN,None
3,Abel Treetops,Abel Treetops was a wizard from Cincinnati who...,NaN,None
4,Euan Abercrombie,Euan Abercrombie was a small boy with prominen...,NaN,None
...,...,...,...,...
1252,Young Red-Haired Witch,"In early September of 1927, a young red-haired...",NaN,None
1253,Yusuf Kama,Yusuf Kama was a French wizard of Senegalese a...,NaN,None
1254,Yvonne,Yvonne was a friend of Petunia Dursley who was...,NaN,None
1255,Zograf,Zograf played Keeper for the Bulgarian Nationa...,NaN,None


In [15]:
dot0 = []
dot1 = []
for i in chars.char_name:
    k = i.split()
    if len(k)>=2: 
        if k[0].endswith('.') :
            dot0.append(i)
        elif k[1].endswith('.'):
            dot1.append(i)

In [ ]:
dot0

In [ ]:
dot1

In [ ]:
# for i in range(chars.shape[0]):
#     k = chars.iloc[i].char_name.split()
#     if len(k)<=2 :
#         chars.iloc[i].firstname = k[0]

In [ ]:
for i in chars.char_name:
    if '.' in i :
        print(i)

In [ ]:
for i in chars.char_name:
    if '.' not in i :
        print(i)

###### After giving up on the above data

### Web Scrapping Part-2

* Here I used a web page with that contained a limited number of character
* I used the google search and another website

In [24]:
page_url = "https://www.google.com/search?q=harry+potter+characters+list&spell=1&sa=X&ved=2ahUKEwi60Nytq7_6AhWBSmwGHebRC7wQBSgAegQIAxA1&biw=1280&bih=573&dpr=1.5"

driver.get(page_url)

In [35]:
gg_chars = driver.find_elements(By.XPATH, "//div[@class='bVj5Zb FozYP']")

In [29]:
page_url = "https://fantasytopics.com/harry-potter-characters-list/"
driver.get(page_url)

In [30]:
imp_chars = driver.find_elements(By.XPATH, "//p[@class='has-text-align-center']")

### Data Preprocessing Part-2

* After getting the character names, I added both the list contents
* Removed duplicates
* Converted the list into a dataframe
* Created a new column called firstname for the character's firstname
* Finally, saved the dataframe into a csv file

In [ ]:
gg_chars

In [37]:
gg_chars_list = [o.text for o in gg_chars]

In [ ]:
gg_chars_list

In [32]:
len(imp_chars)

198

In [33]:
imp_chars_list = [o.text for o in imp_chars]

In [39]:
imp_chars_list = imp_chars_list[1:]

In [43]:
final_character_list = gg_chars_list + imp_chars_list

In [46]:
len(final_character_list)

248

In [ ]:
set([x for x in final_character_list if final_character_list.count(x) > 1])

In [54]:
fl = [*set(final_character_list)] 
fl.pop(0)

''

In [56]:
len(fl)

205

In [79]:
fn = []
for i in fl:
    k = i.split()
    fn.append(k[0])

In [80]:
chs = {'CharacterName':fl,'FirstName':fn}
final_df = pd.DataFrame(chs)

In [82]:
final_df.to_csv('characters.csv',index=False)

* Jump to the extracting relationship notebook